<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/DeepAR_Electricity_HPO_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install mxnet and gluonts

In [0]:
pip install --upgrade mxnet==1.6 gluonts

## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## m4_hourly data loading

In [7]:
dataset = get_dataset("m4_hourly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/test/data.json


In [0]:
new_test_ds = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=dataset.metadata.freq)

## HPO 

In [8]:
pip install --upgrade hyperopt

     |████████████████████████████████| 972kB 4.5MB/s 
  Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [0]:
# choice 탐색 공간
space_num_layers = (2, 3)
space_num_cells= (30, 40, 50)
space_embedding_dimension = (None, [10], [20])
space_cell_type = ('lstm', 'gru')

In [0]:
from hyperopt import hp
from gluonts.evaluation import Evaluator

# 탐색할 공간을 정의
space = {
    'num_layers': hp.choice('nl', space_num_layers),
    'num_cells': hp.choice('nc', space_num_cells),
    'embedding_dimension': hp.choice('ed', space_embedding_dimension),
    'cell_type': hp.choice('ct', space_cell_type),
    'learning_rate': hp.uniform('lr', 0.0001, 0.001),
}

# 테스트할 모델과 평가할 값을 정의
def objective(args):
    num_layers = args['num_layers']
    num_cells = args['num_cells']
    embedding_dimension = args['embedding_dimension']
    cell_type = args['cell_type']
    learning_rate = args['learning_rate']
    
    kwargs = {
        'freq': dataset.metadata.freq,
        'prediction_length': dataset.metadata.prediction_length,
        'trainer': Trainer(ctx="cpu", learning_rate=learning_rate, epochs=1, hybridize=False),
        'num_layers': num_layers,
        'num_cells': num_cells,
        'use_feat_static_cat': True,
        'embedding_dimension': embedding_dimension,
        'cardinality': [int(dataset.metadata.feat_static_cat[0].cardinality)],
        'cell_type': cell_type,
    }
    Estimator = DeepAREstimator(**kwargs)
    predictor = Estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset.test,  #dataset.test,  # test dataset
        predictor=predictor,  # predictor
        num_samples=100,  # number of sample paths we want for evaluation
    )
    
    evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
    agg_metrics, item_metrics = evaluator(ts_it, forecast_it, num_series=len(dataset.test))
    
    return agg_metrics['RMSE']

In [42]:
# minimize the objective over the space
import hyperopt
from hyperopt import fmin, tpe
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

print(best)

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/50 [00:00<?, ?it/s]

 42%|####2     | 21/50 [00:10<00:14,  2.07it/s, epoch=1/1, avg_epoch_loss=6.77]

 86%|########6 | 43/50 [00:20<00:03,  2.08it/s, epoch=1/1, avg_epoch_loss=5.91]

Running evaluation:   0%|          | 0/414 [00:00<?, ?it/s]

Running evaluation:  16%|#5        | 65/414 [00:11<01:03,  5.52it/s]

Running evaluation:  39%|###8      | 161/414 [00:23<00:41,  6.15it/s]

Running evaluation:  62%|######2   | 257/414 [00:34<00:23,  6.70it/s]

Running evaluation:  62%|######2   | 257/414 [00:46<00:23,  6.70it/s]

Running evaluation:  85%|########5 | 353/414 [00:46<00:08,  7.12it/s]

Running evaluation: 100%|##########| 414/414 [00:49<00:00,  8.33it/s]


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
  1%|          | 1/100 [01:16<2:05:55, 76.32s/trial, best loss: 7978.736308391147]

  0%|          | 0/50 [00:00<?, ?it/s]

 42%|####2     | 21/50 [00:10<00:14,  2.02it/s, epoch=1/1, avg_epoch_loss=6.61]

 84%|########4 | 42/50 [00:20<00:03,  2.03it/s, epoch=1/1, avg_epoch_loss=5.89]

Running evaluation:   0%|          | 0/414 [00:00<?, ?it/s]

Running evaluation:  16%|#5        | 65/414 [00:12<01:09,  5.03it/s]

Running evaluation:  39%|###8      | 161/414 [00:25<00:45,  5.56it/s]

Running evaluation:  62%|######2   | 257/414 [00:38<00:25,  6.05it/s]

Running evaluation:  62%|######2   | 257/414 [00:49<00:25,  6.05it/s]

Running evaluation:  85%|########5 | 353/414 [00:51<00:09,  6.45it/s]

Running evaluation: 100%|##########| 414/414 [00:55<00:00,  7.52it/s]


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
  2%|▏         | 2/100 [02:38<2:07:35, 78.12s/trial, best loss: 6734.00741294185]

...

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
 98%|█████████▊| 98/100 [2:30:49<03:10, 95.34s/trial, best loss: 3291.1304598623396]

  0%|          | 0/50 [00:00<?, ?it/s]

 30%|###       | 15/50 [00:10<00:24,  1.43it/s, epoch=1/1, avg_epoch_loss=5.93]

 62%|######2   | 31/50 [00:21<00:13,  1.45it/s, epoch=1/1, avg_epoch_loss=5.73]

 94%|#########3| 47/50 [00:31<00:02,  1.47it/s, epoch=1/1, avg_epoch_loss=5.73]

Running evaluation:   0%|          | 0/414 [00:00<?, ?it/s]

Running evaluation:  16%|#5        | 65/414 [00:13<01:10,  4.92it/s]

Running evaluation:  39%|###8      | 161/414 [00:26<00:46,  5.47it/s]

Running evaluation:  39%|###8      | 161/414 [00:38<00:46,  5.47it/s]

Running evaluation:  62%|######2   | 257/414 [00:39<00:26,  5.94it/s]

Running evaluation:  85%|########5 | 353/414 [00:51<00:09,  6.33it/s]

Running evaluation: 100%|##########| 414/414 [00:56<00:00,  7.38it/s]


learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.
 99%|█████████▉| 99/100 [2:32:21<01:34, 94.45s/trial, best loss: 3291.1304598623396]

  0%|          | 0/50 [00:00<?, ?it/s]

 44%|####4     | 22/50 [00:10<00:13,  2.10it/s, epoch=1/1, avg_epoch_loss=6.23]

 88%|########8 | 44/50 [00:20<00:02,  2.10it/s, epoch=1/1, avg_epoch_loss=5.97]

Running evaluation:   0%|          | 0/414 [00:00<?, ?it/s]

Running evaluation:  16%|#5        | 65/414 [00:13<01:15,  4.64it/s]

Running evaluation:  16%|#5        | 65/414 [00:26<01:15,  4.64it/s]

Running evaluation:  39%|###8      | 161/414 [00:27<00:49,  5.16it/s]

Running evaluation:  62%|######2   | 257/414 [00:41<00:28,  5.60it/s]

Running evaluation:  85%|########5 | 353/414 [00:55<00:10,  5.96it/s]

Running evaluation: 100%|##########| 414/414 [00:59<00:00,  6.95it/s]


100%|██████████| 100/100 [2:33:47<00:00, 92.27s/trial, best loss: 3291.1304598623396]
{'ct': 0, 'ed': 1, 'lr': 0.0006028651142341776, 'nc': 2, 'nl': 0}


In [43]:
best

{'ct': 0, 'ed': 1, 'lr': 0.0006028651142341776, 'nc': 2, 'nl': 0}

### 탐색된 값을 이용해서 모델 학습

In [0]:
    kwargs = {
        'freq': dataset.metadata.freq,
        'prediction_length': dataset.metadata.prediction_length,
        'trainer': Trainer(ctx="cpu", learning_rate=best['lr'], epochs=100, hybridize=False),
        'num_layers': space_num_layers[best['nl']],
        'num_cells': space_num_cells[best['nc']],
        'use_feat_static_cat': True,
        'embedding_dimension': space_embedding_dimension[best['ed']],
        'cardinality': [int(dataset.metadata.feat_static_cat[0].cardinality)],
        'cell_type': space_cell_type[best['ct']],
    }
    Estimator = DeepAREstimator(**kwargs)

In [45]:
predictor = Estimator.train(dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:23<00:00,  2.17it/s, epoch=100/100, avg_epoch_loss=3.45]


In [46]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test, 
    predictor=predictor, 
    num_samples=100, 
)
forecasts = list(forecast_it)
tss = list(ts_it)
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(dataset.test))
print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 414/414 [00:00<00:00, 13959.32it/s]


{
    "MSE": 11404481.354284063,
    "abs_error": 10111671.245807648,
    "abs_target_sum": 145558863.59960938,
    "abs_target_mean": 7324.822041043147,
    "seasonal_error": 336.9046924038302,
    "MASE": 2.333333654873358,
    "MAPE": 0.14001955869578406,
    "sMAPE": 0.13698648541067157,
    "OWA": NaN,
    "MSIS": 27.2664757790818,
    "QuantileLoss[0.1]": 3954275.5656981473,
    "Coverage[0.1]": 0.047151771336553905,
    "QuantileLoss[0.5]": 10111671.094428062,
    "Coverage[0.5]": 0.21401972624798715,
    "QuantileLoss[0.9]": 5814418.021376991,
    "Coverage[0.9]": 0.6522745571658611,
    "RMSE": 3377.05216931632,
    "NRMSE": 0.4610422137758019,
    "ND": 0.0694679183098183,
    "wQuantileLoss[0.1]": 0.027166161289739275,
    "wQuantileLoss[0.5]": 0.06946791726982951,
    "wQuantileLoss[0.9]": 0.039945475511342164,
    "mean_wQuantileLoss": 0.04552651802363699,
    "MAE_Coverage": 0.19551798174986593
}
